In [1]:
import numpy as np
import pandas as pd

#parametrii modelului liniar
np.random.seed(42)  #pt reproductibilitate
num_points = 100
m_true = 1500  #panta
b_true = 2000  #interceptul
sigma_noise = 500  #dev std a zgomotului

#generare variabila independenta (ani de experienta)
x = np.random.uniform(0, 20, size=num_points)

#calc veniturile observate cu zgomot
noise = np.random.normal(0, sigma_noise, size=num_points)
y = m_true * x + b_true + noise

#salvam datele într-un DataFrame pentru vizualizare ulterioara
data = pd.DataFrame({"Experienta (ani)": x, "Venit lunar (RON)": y})

data.head()


,Experienta (ani),Venit lunar (RON)
0,7.490802,13279.727100
1,19.014286,30371.925517
2,14.639879,24005.698643
3,11.973170,18965.970069
4,3.120373,6570.723269


In [2]:
import pymc as pm
import arviz as az

#definim modelul Bayesian
with pm.Model() as linear_model:
    #priori pentru parametrii m și b
    m = pm.Normal("m", mu=0, sigma=1000)
    b = pm.Normal("b", mu=0, sigma=1000)

    #priori pentru sigma (zgomotul)
    sigma = pm.HalfNormal("sigma", sigma=1000)

    #rel liniara
    y_est = m * x + b

    #verosimilitatea
    y_obs = pm.Normal("y_obs", mu=y_est, sigma=sigma, observed=y)

    #mostrare a posteriori
    trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=42)

#rezumatul parametrilor a posteriori
summary = az.summary(trace, round_to=2)
summary


Output()

Output()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b,2094.11,83.61,1943.36,2252.54,1.87,1.33,2020.63,2057.20,1.0
m,1489.44,7.66,1474.74,1503.03,0.19,0.13,1684.94,1599.38,1.0
sigma,458.78,32.79,395.40,518.08,0.66,0.47,2465.88,2502.72,1.0


In [3]:
#reducem numarul de mostre si reglam nr de iterari de tuning pentru performanta
with pm.Model() as linear_model_fast:
    #priori pentru parametrii m și b
    m = pm.Normal("m", mu=0, sigma=1000)
    b = pm.Normal("b", mu=0, sigma=1000)

    #priori pentru sigma (zgomotul)
    sigma = pm.HalfNormal("sigma", sigma=1000)

    #rel liniara
    y_est = m * x + b

    #verosimilitatea
    y_obs = pm.Normal("y_obs", mu=y_est, sigma=sigma, observed=y)

    #mostrare a posteriori cu mai putine iteratii
    trace_fast = pm.sample(1000, tune=500, return_inferencedata=True, random_seed=42)

#rezumatul parametrilor a posteriori
summary_fast = az.summary(trace_fast, round_to=2)
summary_fast


Output()

Output()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b,2090.49,85.79,1934.87,2250.74,2.83,2.00,920.55,1135.09,1.0
m,1489.90,7.82,1475.66,1504.20,0.25,0.18,950.71,993.00,1.0
sigma,458.95,32.87,396.92,518.82,1.04,0.74,997.37,750.81,1.0
